In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    global modelo
    
    # Obtener el número de pasos del formulario
    steps = request.form.get('steps')
    
    # Obtener el archivo CSV exógeno del formulario
    file = request.files['data']
    
    # Verificar si todos los campos requeridos están presentes
    if None in (steps, file):
        return jsonify({'error': 'Faltan campos requeridos en el formulario.'}), 400
    
    # Convertir los pasos a un entero
    steps = int(steps)
    
    # Leer el archivo CSV como DataFrame
    try:
        df = pd.read_csv(file)
        df = pd.DataFrame(df)
        
        # Convertir el índice del DataFrame exógeno a DatetimeIndex
        df.index = pd.to_datetime(df.index)
        df = df[~df.index.duplicated()]

        # Now, reindex with hourly frequency
        df = df.asfreq('h')
        df.fillna(0, inplace=True)
        
    except Exception as e:
        return jsonify({'error': 'Error al leer el archivo CSV: ' + str(e)}), 400
    
    # Cargar el modelo si no se ha cargado aún
    if modelo is None:
        cargar_modelo()
    
    if modelo is not None:
        # Ahora, utiliza el número de pasos recibido y los datos exógenos para realizar la predicción
        predictions = modelo.predict(steps=steps, exog=df[['GHI', 'Festivo', 'Gas', 'PotenciaViento']])
        # Devolver los resultados como una respuesta JSON
        return jsonify({'predictions': list(predictions)})
    else:
        return jsonify({'error': 'El modelo no pudo ser cargado.'}), 500

In [2]:

from flask import Flask, request, render_template,jsonify
import os
import seaborn as sns
import pandas as pd
from joblib import  load
import statsmodels.api as sm

In [15]:
df = pd.read_csv(r'C:\Users\laura\OneDrive\Desktop\PROYECTO ML\ml_project\src\data\processed\test.csv')
df = pd.DataFrame(df)

In [16]:
df

,datetime,GHI,Festivo,Gas,PotenciaViento,Precios,year,month,day,hour,GMT
0,2022-11-22 08:00:00,3143.000,0.0,54.0,192818.000,100.00,2022.0,11.0,22.0,8.0,1.0
1,2022-11-22 09:00:00,20261.000,0.0,54.0,198693.000,99.63,2022.0,11.0,22.0,9.0,1.0
2,2022-11-22 10:00:00,38988.000,0.0,54.0,206052.000,84.20,2022.0,11.0,22.0,10.0,1.0
3,2022-11-22 11:00:00,45054.000,0.0,54.0,214367.000,66.37,2022.0,11.0,22.0,11.0,1.0
4,2022-11-22 12:00:00,46024.000,0.0,54.0,220386.000,62.70,2022.0,11.0,22.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6707,2023-08-28 19:00:00,8086.000,0.0,72.0,11135.500,108.49,2023.0,8.0,28.0,19.0,2.0
6708,2023-08-28 20:00:00,2393.083,0.0,72.0,11101.333,133.52,2023.0,8.0,28.0,20.0,2.0
6709,2023-08-28 21:00:00,782.750,0.0,72.0,10576.333,138.08,2023.0,8.0,28.0,21.0,2.0
6710,2023-08-28 22:00:00,674.083,0.0,72.0,10107.167,123.72,2023.0,8.0,28.0,22.0,2.0


In [17]:
df.set_index('datetime', inplace=True)
df = df[~df.index.duplicated()]

In [25]:
df.index.type

AttributeError: 'DatetimeIndex' object has no attribute 'type'

In [18]:
df

,GHI,Festivo,Gas,PotenciaViento,Precios,year,month,day,hour,GMT
datetime,,,,,,,,,,
2022-11-22 08:00:00,3143.000,0.0,54.0,192818.000,100.00,2022.0,11.0,22.0,8.0,1.0
2022-11-22 09:00:00,20261.000,0.0,54.0,198693.000,99.63,2022.0,11.0,22.0,9.0,1.0
2022-11-22 10:00:00,38988.000,0.0,54.0,206052.000,84.20,2022.0,11.0,22.0,10.0,1.0
2022-11-22 11:00:00,45054.000,0.0,54.0,214367.000,66.37,2022.0,11.0,22.0,11.0,1.0
2022-11-22 12:00:00,46024.000,0.0,54.0,220386.000,62.70,2022.0,11.0,22.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2023-08-28 19:00:00,8086.000,0.0,72.0,11135.500,108.49,2023.0,8.0,28.0,19.0,2.0
2023-08-28 20:00:00,2393.083,0.0,72.0,11101.333,133.52,2023.0,8.0,28.0,20.0,2.0
2023-08-28 21:00:00,782.750,0.0,72.0,10576.333,138.08,2023.0,8.0,28.0,21.0,2.0


In [19]:
        # Now, reindex with hourly frequency
df = df.asfreq('h')


In [21]:
df.fillna(0, inplace=True)

In [24]:
df

,GHI,Festivo,Gas,PotenciaViento,Precios,year,month,day,hour,GMT
datetime,,,,,,,,,,
2022-11-22 08:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-22 09:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-22 10:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-22 11:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-22 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2023-08-28 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-08-28 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-08-28 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df